In [1]:
import re
from collections import Counter
from typing import List, Tuple

# from pymorphy2 import MorphAnalyzer
from bedarev_analyzer import MorphAnalyzer

In [2]:
with open("./data/clear_text.txt", 'r', encoding='utf8') as fp:
    text = fp.read()

In [3]:
class NGrammer():
    morph = MorphAnalyzer()
    DELIMITERS = (',', '.', '!', '?', ':')
    
    def __init__(self, text, min_count = 2):
        for delimiter in self.DELIMITERS:
            text = text.replace(delimiter, '\n')
        normal_text = [l for l in (self.normalize(line) for line in text.split('\n')) if l]
        self.min_count = max(2, min_count)
        self.dicts_all_n_gramms = []

        for n in range(1, max([len(l) for l in normal_text]) + 1):
            all_n_gramms = []
            for normal_line in normal_text:
                if len(normal_line) < n:
                    continue
                n_gramms = self.get_n_gramm(normal_line, n)
                if n_gramms:
                        all_n_gramms.extend(n_gramms)
            dict_all_n_gramms = {name: count for name, count in Counter(all_n_gramms).items() if name and count >= self.min_count}
            if not dict_all_n_gramms:
                break
            self.dicts_all_n_gramms.append(dict_all_n_gramms)

    @classmethod
    def normalize(cls, line: str) -> List[str]:
        return [w for w in [cls.morph.parse(word)[0].normal_form for word in
                (v.lower() for v in re.findall(r"('?[а-яА-ЯёЁ][а-яА-ЯёЁ]*(?:-[а-яА-ЯёЁ]+)*'?)", line))] if w and w != ' ']

    @staticmethod
    def get_n_gramm(words: list, n: int = 2): 
        return [tuple(words[i-n+1:i+1]) for i in range(n - 1, len(words))]

    def max_n(self):
        return len(self.dicts_all_n_gramms) - 1

    def gramms(self, n=None):
        result = Counter()
        if n is None:
            for i in self.dicts_all_n_gramms[1:]:
                result += Counter(i)
        else:
            result = Counter(self.dicts_all_n_gramms[n-1])
        result = Counter({' '.join(key): value for key, value in result.items()})
        return result

In [4]:
grammer = NGrammer(text)

In [5]:
grammer.gramms().most_common(10)

[('в работа', 872),
 ('на основа', 499),
 ('в г', 492),
 ('а также', 425),
 ('мочь быть', 421),
 ('с помощь', 382),
 ('при этом', 373),
 ('а в', 359),
 ('в который', 334),
 ('связанный с', 332)]

In [6]:
grammer.gramms(3).most_common(10)

[('в данный работа', 162),
 ('в настоящее время', 157),
 ('в связь с', 145),
 ('по сравнение с', 143),
 ('в то время', 123),
 ('в зависимость от', 106),
 ('в соответствие с', 104),
 ('в этом случай', 101),
 ('в настоящий работа', 91),
 ('с точка зрение', 84)]

In [8]:
print(grammer.max_n())
grammer.gramms(grammer.max_n()).most_common(10)

41


[('сотрудник который ученик евграфа сергеевича кузнецова принимать активный участие в покорение космос открытие космический эра летию первое инструментальный оптический исследование земной атмосфера из космос с пилотировать космический корабль ифа ан ссср миэиа мап с участие институт келдыша летию первое научный эксперимент',
  2),
 ('который ученик евграфа сергеевича кузнецова принимать активный участие в покорение космос открытие космический эра летию первое инструментальный оптический исследование земной атмосфера из космос с пилотировать космический корабль ифа ан ссср миэиа мап с участие институт келдыша летию первое научный эксперимент по',
  2)]